In [1]:
from qdrant_client import QdrantClient
from extractors.markdown import MarkdownLoader
from ingest.qdrant_etl import QdrantFastEmbedETLService

In [2]:
qdrant_host = "localhost"
qdrant_port = 6333
embedding_models_dir = "C:\\Users\\Nilanjan\\Models"
embedding_threads = 16
sparse_vec_model_name = "Qdrant/bm25"
dense_vec_model_name = "intfloat/multilingual-e5-large"
late_interaction_model_name = "jinaai/jina-colbert-v2"

In [3]:
etl_service = QdrantFastEmbedETLService(qdrant_host=qdrant_host,
                                        qdrant_port=qdrant_port,
                                        embedding_models_dir=embedding_models_dir,
                                        sparse_vec_model_name=sparse_vec_model_name,
                                        dense_vec_model_name=dense_vec_model_name,
                                        late_interaction_model_name=late_interaction_model_name,
                                        embedding_threads=embedding_threads)

C:\Users\Nilanjan\Repos\hybrid_search_rag\ingest\qdrant_etl.py:39: UserWarning: The model intfloat/multilingual-e5-large now uses mean pooling instead of CLS embedding. In order to preserve the previous behaviour, consider either pinning fastembed version to 0.5.1 or using `add_custom_model` functionality.
  self.dense_embedding_model = TextEmbedding(


### Create a Collection

In [4]:
dense_embedding_size = 1024
late_interaction_embedding_size = 128
collection_name= 'hybrid_search'

etl_service.create_hybrid_search_collection(collection_name,
                                            dense_embedding_size,
                                            late_interaction_embedding_size)

Collection 'hybrid_search' created.


### Load and chunk Documents

In [5]:
loader = MarkdownLoader()

In [7]:
directory = "C:\\Users\\Nilanjan\\Documents\\njkol-workspace\\2.Areas and Resources\\Lifestyle"
documents = loader.load_docs(directory)
chunked_docs = loader.split_docs(documents)

  0%|                                                                                                                                        | 0/48 [00:00<?, ?it/s]libmagic is unavailable but assists in filetype detection. Please consider installing libmagic for better results.
libmagic is unavailable but assists in filetype detection. Please consider installing libmagic for better results.
libmagic is unavailable but assists in filetype detection. Please consider installing libmagic for better results.
libmagic is unavailable but assists in filetype detection. Please consider installing libmagic for better results.
libmagic is unavailable but assists in filetype detection. Please consider installing libmagic for better results.
libmagic is unavailable but assists in filetype detection. Please consider installing libmagic for better results.
libmagic is unavailable but assists in filetype detection. Please consider installing libmagic for better results.
libmagic is unavailable but ass

### Generate embeddings & upload to Vector Store

In [8]:
## Create a Points with Multi-Vector Representation
points  = etl_service.create_multivector_points_from_documents(chunked_docs)

Embedding process completed in 137.78 seconds


In [9]:
### Upload points in batch in a parallel manner
collection_name= 'hybrid_search'
max_threads = 8
max_retries = 3

etl_service.upload_points(collection_name, points, max_threads, max_retries)

Uploading points completed in 9.20 seconds
